In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

Configurações básicas

In [14]:
device = torch.device("dml") if torch.cuda.is_available() else torch.device("cpu")
batch_size = 32
num_epochs = 10
learning_rate = 0.001
val_ratio = 0.2

Transformação dos dados

In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Carregar os dados

In [16]:
dataset = datasets.ImageFolder("./train", transform=transform)

In [17]:
train_size = int((1 - val_ratio) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [18]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))  # Ajustar para o número de classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Treinamento

In [19]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {running_loss/len(train_loader):.4f}")

    # Validação
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch 1/10, Train Loss: 1.7862
Epoch 1/10, Val Loss: 2.4615, Val Accuracy: 46.86%
Epoch 2/10, Train Loss: 1.1328
Epoch 2/10, Val Loss: 1.5510, Val Accuracy: 57.71%
Epoch 3/10, Train Loss: 0.6658
Epoch 3/10, Val Loss: 1.5953, Val Accuracy: 54.29%
Epoch 4/10, Train Loss: 0.3619
Epoch 4/10, Val Loss: 2.3588, Val Accuracy: 36.57%
Epoch 5/10, Train Loss: 0.4650
Epoch 5/10, Val Loss: 1.9164, Val Accuracy: 48.00%
Epoch 6/10, Train Loss: 0.4354
Epoch 6/10, Val Loss: 1.9259, Val Accuracy: 56.00%
Epoch 7/10, Train Loss: 0.3654
Epoch 7/10, Val Loss: 2.1598, Val Accuracy: 56.57%
Epoch 8/10, Train Loss: 0.2319
Epoch 8/10, Val Loss: 2.3549, Val Accuracy: 46.86%
Epoch 9/10, Train Loss: 0.1194
Epoch 9/10, Val Loss: 1.7416, Val Accuracy: 56.57%
Epoch 10/10, Train Loss: 0.0812
Epoch 10/10, Val Loss: 1.9244, Val Accuracy: 55.43%


Salvando o modelo

In [20]:
model_path = "resnet18_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Modelo salvo em: {model_path}")

Modelo salvo em: resnet18_model.pth


Resultados

In [21]:
print("Treinamento concluído.")

Treinamento concluído.
